# Day 3: Disney Parks Guest Spending Behavior

You are a data analyst working with the Disney Parks revenue team to understand nuanced guest spending patterns across different park experiences. The team wants to develop a comprehensive view of visitor purchasing behaviors. Your goal is to uncover meaningful insights that can drive personalized marketing strategies.

In [ ]:
import pandas as pd
import numpy as np

fct_guest_spending_data = [
  {
    "guest_id": 1,
    "visit_date": "2024-07-05",
    "amount_spent": 50,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-07-06",
    "amount_spent": 30,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-07-10",
    "amount_spent": 20.5,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 4,
    "visit_date": "2024-07-12",
    "amount_spent": 40,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-07-15",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 5,
    "visit_date": "2024-07-20",
    "amount_spent": 60,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 6,
    "visit_date": "2024-07-25",
    "amount_spent": 25,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-03",
    "amount_spent": 55,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-15",
    "amount_spent": 45,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-05",
    "amount_spent": 22,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-20",
    "amount_spent": 38,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 7,
    "visit_date": "2024-08-10",
    "amount_spent": 15,
    "park_experience_type": "Character Meet"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-25",
    "amount_spent": 28,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-27",
    "amount_spent": 32,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-02",
    "amount_spent": 65,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-05",
    "amount_spent": 50,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 9,
    "visit_date": "2024-09-15",
    "amount_spent": 40,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 10,
    "visit_date": "2024-09-20",
    "amount_spent": 70,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-25",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-28",
    "amount_spent": 10,
    "park_experience_type": "Character Meet"
  }
]
fct_guest_spending = pd.DataFrame(fct_guest_spending_data)


## Question 1

What is the average spending per guest per visit for each park experience type during July 2024? Ensure that park experience types with no recorded transactions are shown with an average spending of 0.0. This analysis helps establish baseline spending differences essential for later segmentation.

In [ ]:
# Note: pandas and numpy are already imported as pd and np
# The following tables are loaded as pandas DataFrames with the same names: fct_guest_spending
# Please print your final result or dataframe
unique_park_experience_types = fct_guest_spending['park_experience_type'].drop_duplicates()
filtered_df = fct_guest_spending[
  fct_guest_spending['visit_date'].between('2024-07-01','2024-07-31')
  ]

avg_spending=filtered_df.groupby(['park_experience_type'])['amount_spent'].mean().reset_index(name = 'avg_spent')

final_result = (
    pd.merge(unique_park_experience_types,avg_spending, on='park_experience_type', how='left')
    .fillna({'avg_spent': 0.0})
)
final_result

## Question 2

For guests who visited our parks more than once in August 2024, what is the difference in spending between their first and their last visit? This investigation, using sequential analysis, will reveal any shifts in guest spending behavior over multiple visits.

In [ ]:
# Step 1: Filter for August 2024
filtered_df = fct_guest_spending[
    fct_guest_spending['visit_date'].between('2024-08-01', '2024-08-31')
][['guest_id', 'visit_date', 'amount_spent']]

# Step 2: Count visits per guest
visit_counts = filtered_df['guest_id'].value_counts()
repeat_visitors = visit_counts[visit_counts > 1].index

# Step 3: Keep only repeat visitors
repeat_df = filtered_df[filtered_df['guest_id'].isin(repeat_visitors)]

# Step 4: Aggregate amount_spent per guest per date
daily_spend = repeat_df.groupby(['guest_id', 'visit_date'])['amount_spent'].sum().reset_index()

# Step 5: Get first and last visit spending per guest
first_visits = daily_spend.sort_values('visit_date').groupby('guest_id').first().rename(columns={'amount_spent': 'first_spend'})
last_visits = daily_spend.sort_values('visit_date').groupby('guest_id').last().rename(columns={'amount_spent': 'last_spend'})

# Step 6: Combine and calculate difference
visit_diff = first_visits[['first_spend']].join(last_visits[['last_spend']])
visit_diff['spend_diff'] = visit_diff['last_spend'] - visit_diff['first_spend']

# Step 7: Reset index and display
visit_diff = visit_diff.reset_index()
print(visit_diff)

## Question 3

In September 2024, how can guests be categorized into distinct spending segments such as Low, Medium, and High based on their total spending? Use the following thresholds for categorization: 
-Low: Includes values from $0 up to, but not including, $50.
-Medium: Includes values from $50 up to, but not including, $100.
-High: Includes values from $100 and above. 
Exclude guests who did not make any purchases in the period.

In [ ]:
# Step 1: Filter data for September 2024
filtered_df = fct_guest_spending[
    fct_guest_spending['visit_date'].between('2024-09-01', '2024-09-30')
]

# Step 2: Compute total amount spent per guest
total_spending = filtered_df.groupby('guest_id')['amount_spent'].sum().reset_index(name='Total_amount_spent')

# Step 3: Exclude guests with 0 total spending (if any)
total_spending = total_spending[total_spending['Total_amount_spent'] > 0]

# Step 4: Define segmentation function
def guest_segment(x):
    if x < 50.0:
        return "Low"
    elif x < 100.0:
        return "Medium"
    else:
        return "High"

# Step 5: Apply function to create segment column
total_spending['spending_segment'] = total_spending['Total_amount_spent'].map(guest_segment)

# Step 6: View the result
print(total_spending)

Made with ❤️ by [Interview Master](https://www.interviewmaster.ai)